# LeNet5 applied to the MNIST dataset




In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the LeNet-5 model
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = x.view(-1, 120)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 100
max_iterations = 10000

# Prepare dataset and dataloaders
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader1 = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader1 = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model1 = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=learning_rate)

# Training  loop
def model_train():
    iteration = 0
    for epoch in range(num_epochs):
        model1.train()
        running_loss = 0.0

        for i, (inputs, labels) in enumerate(train_loader1):
            if iteration >= max_iterations:
                print(f"Stopping early after {max_iterations} iterations")
                model_test()
                return

            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model1(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            iteration += 1

            if i % (len(train_loader1)/4) == 0:  # Print 4 times every Epoch
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader1)}], Loss: {running_loss / 100:.4f}")
                running_loss = 0.0

        print(f"Epoch [{epoch+1}/{num_epochs}]")
        model_test()


# Testing loop
def model_test():
    model1.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader1:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model1(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")



In [ ]:
# Run training and testing
model_train()

Epoch [1/100], Step [1/938], Loss: 0.0231
Epoch [1/100], Step [470/938], Loss: 2.4652
Epoch [1/100]
Test Accuracy: 96.26%
Epoch [2/100], Step [1/938], Loss: 0.0012
Epoch [2/100], Step [470/938], Loss: 0.4447
Epoch [2/100]
Test Accuracy: 97.72%
Epoch [3/100], Step [1/938], Loss: 0.0001
Epoch [3/100], Step [470/938], Loss: 0.3114
Epoch [3/100]
Test Accuracy: 97.99%
Epoch [4/100], Step [1/938], Loss: 0.0004
Epoch [4/100], Step [470/938], Loss: 0.2127
Epoch [4/100]
Test Accuracy: 98.56%
Epoch [5/100], Step [1/938], Loss: 0.0001
Epoch [5/100], Step [470/938], Loss: 0.1760
Epoch [5/100]
Test Accuracy: 98.66%
Epoch [6/100], Step [1/938], Loss: 0.0001
Epoch [6/100], Step [470/938], Loss: 0.1733
Epoch [6/100]
Test Accuracy: 98.73%
Epoch [7/100], Step [1/938], Loss: 0.0001
Epoch [7/100], Step [470/938], Loss: 0.1277
Epoch [7/100]
Test Accuracy: 99.02%
Epoch [8/100], Step [1/938], Loss: 0.0006
Epoch [8/100], Step [470/938], Loss: 0.1137
Epoch [8/100]
Test Accuracy: 99.01%
Epoch [9/100], Step [1/9

# ResNet-10 applied to the CIFAR-10 dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define the basic block used in ResNet-10
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# Define the ResNet-10 model
class ResNet10(nn.Module):
    def __init__(self):
        super(ResNet10, self).__init__()
        self.in_channels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        # Stack of blocks
        self.layer1 = self._make_layer(16, 1, stride=1)
        self.layer2 = self._make_layer(32, 1, stride=2)
        self.layer3 = self._make_layer(64, 1, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 10)

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Load CIFAR-10 data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader2 = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader2 = torch.utils.data.DataLoader(testset, batch_size=256, shuffle=False, num_workers=2)

# Initialize the model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model2 = ResNet10().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model2.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model2.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader2, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch {epoch+1}, Batch {i+1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

Epoch 1, Batch 100, Loss: 0.495
Epoch 2, Batch 100, Loss: 0.478
Epoch 3, Batch 100, Loss: 0.465
Epoch 4, Batch 100, Loss: 0.448
Epoch 5, Batch 100, Loss: 0.424
Epoch 6, Batch 100, Loss: 0.421
Epoch 7, Batch 100, Loss: 0.409
Epoch 8, Batch 100, Loss: 0.391
Epoch 9, Batch 100, Loss: 0.378
Epoch 10, Batch 100, Loss: 0.366
Finished Training


In [ ]:
# Evaluate the model
model2.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader2:
        images, labels = data
        outputs = model2(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.cpu() == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 73.25%
